In [10]:
import pickle
import numpy as np
from dataset import Dataset
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, ElasticNet, SGDRegressor, BayesianRidge

In [2]:
with open("lpf_sensor_data.pkl", "rb") as f:
    sensor_data = pickle.load(f)

with open("sensor_labels.pkl", "rb") as f:
    labels = pickle.load(f)

with open("interpolation_functions.pkl", "rb") as f:
    interp_funcs = pickle.load(f)

dataset = Dataset(sensor_data, labels, interp_funcs)

In [12]:
rs = 42
results = {
    "mat_0": {"s_0_1": [], "s_2_3": [], "s_4_5": [], "s_6_7": []},
    "mat_1": {"s_0_1": [], "s_2_3": [], "s_4_5": [], "s_6_7": []}
}
df_dict = {
    "LR": [],
    "GBR": [],
    "EN": [],
    "SGDR": [],
    "SVR": [],
    "BR": []
}

for matrix in range(2):
    for s in range(0, 8, 2):
        s_l, s_r = s, s+1
        X, y, _, targets = dataset.get_sensor_pair_cls(matrix,
                                                       (s_l, s_r),
                                                       num_samples=100,
                                                       as_log=True,
                                                       as_mean=False)
        X, y = dataset.clean_up_regression_data(X, targets)
        X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                            test_size=0.2,
                                                            random_state=rs)
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        model = LinearRegression()
        model = model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        score_r2 = r2_score(y_test, y_pred)
        reg = "LR"
        results[f"mat_{matrix}"][f"s_{s_l}_{s_r}"].append(
            {"reg": reg, "r2": np.round(score_r2, 3)}
        )
        df_dict[reg].append(np.round(score_r2, 3))

        model = GradientBoostingRegressor()
        model = model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        score_r2 = r2_score(y_test, y_pred)
        reg = "GBR"
        results[f"mat_{matrix}"][f"s_{s_l}_{s_r}"].append(
            {"reg": reg, "r2": np.round(score_r2, 3)}
        )
        df_dict[reg].append(np.round(score_r2, 3))

        model = ElasticNet()
        model = model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        score_r2 = r2_score(y_test, y_pred)
        reg = "EN"
        results[f"mat_{matrix}"][f"s_{s_l}_{s_r}"].append(
            {"reg": reg, "r2": np.round(score_r2, 3)}
        )
        df_dict[reg].append(np.round(score_r2, 3))

        model = SGDRegressor()
        model = model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        score_r2 = r2_score(y_test, y_pred)
        reg = "SGDR"
        results[f"mat_{matrix}"][f"s_{s_l}_{s_r}"].append(
            {"reg": reg, "r2": np.round(score_r2, 3)}
        )
        df_dict[reg].append(np.round(score_r2, 3))

        model = SVR()
        model = model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        score_r2 = r2_score(y_test, y_pred)
        reg = "SVR"
        results[f"mat_{matrix}"][f"s_{s_l}_{s_r}"].append(
            {"reg": reg, "r2": np.round(score_r2, 3)}
        )
        df_dict[reg].append(np.round(score_r2, 3))

        model = BayesianRidge()
        model = model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        score_r2 = r2_score(y_test, y_pred)
        reg = "BR"
        results[f"mat_{matrix}"][f"s_{s_l}_{s_r}"].append(
            {"reg": reg, "r2": np.round(score_r2, 3)}
        )
        df_dict[reg].append(np.round(score_r2, 3))

In [13]:
df_dict

{'LR': [0.998, 0.999, 0.999, 1.0, 0.997, 1.0, 0.999, 1.0],
 'GBR': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 'EN': [0.973, 0.934, 0.962, 0.957, 0.976, 0.991, 0.982, 0.98],
 'SGDR': [0.994, 0.996, 0.997, 0.998, 0.984, 0.999, 0.998, 0.999],
 'SVR': [0.994, 0.991, 0.996, 0.996, 0.984, 0.998, 0.987, 0.993],
 'BR': [0.998, 0.999, 0.999, 1.0, 0.997, 1.0, 0.999, 1.0]}

In [14]:
import pandas as pd
df = pd.DataFrame(df_dict)
df.to_excel("regressor_scores.xlsx")